In [ ]:
# The ULP is the ultra low power co-processor on the ESP32 that we use as a realtime 
# co-processor for decoding PWM signals

In [66]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [5]:
# The assembler for this processor machine code runs in Micropython and is at
#    https://github.com/ThomasWaldmann/py-esp32-ulp

%sendtofile --mkdir --source /home/julian/extrepositories/py-esp32-ulp/esp32_ulp  esp32_ulp

Sent 283 lines (9521 bytes) to esp32_ulp/assemble.py.


In [67]:
from esp32_ulp.__main__ import src_to_binary

source = """\
dataiorec:  .long 0x00000000    // ->r3 as address
timepin27:  .long 0x00000000    // r3[4]
countpin27: .long 0x00000000    // r3[8]
divider:    .long 0x0000000F    // r3[12]

entry:
            move r3, dataiorec
outerloop:
            ld   r2, r3, 12     // divider
            move r1, 0x0000     // pin27 counter
ioloop:
            add  r1, r1, 0x0001
            jump outerloop, ov
            reg_rd 0x3ff48424, 16, 17   // read IO register into r0
            st   r0, r3, 0      // dataiorec[0]=r0 (for monitoring)
            rsh  r0, r0, 14     // shift pin 27 to bit0
            add  r0, r0, r2     // add the dividercounter of bit flips
            and  r0, r0, 0x0001 
            jump ioloop, eq

            sub  r2, r2, 0x0001 // decrement the divider
            jump ioloopend, eq
            jump ioloop
ioloopend:
            st   r1, r3, 4      // dataiorec[1]=r1   timepin27
            ld   r0, r3, 8      // r0=dataiorec[2]   countpin27
            add  r0, r0, 1      // r0++
            st   r0, r3, 8      // dataiorec[2]=r0   countpin27
            jump outerloop
            halt
"""
binary = src_to_binary(source)
fout = open("measure_windprop.ulp", "wb")
fout.write(binary)
fout.close()


0000 dataiorec
0001 timepin27
0002 countpin27
0003 divider
0004 entry
0005 outerloop
0007 ioloop
0018 ioloopend


In [63]:
import machine, time, esp32

p27 = machine.Pin(27, machine.Pin.IN, machine.Pin.PULL_DOWN)
print(p27.value())

entry = 4
ulp = esp32.ULP()
ulp.load_binary(0, open("measure_windprop.ulp", "rb").read())
ulp.set_wakeup_period(0, 1000000)
ulp.run(entry*4)
def readulpdata(r):
    return machine.mem32[0x50000000+r]&0xFFFF
def writeulpdata(r, v):
    machine.mem32[0x50000000+r] = v&0xFFFF


0


In [64]:
#w = machine.PWM(machine.Pin(5))
#w.freq(50)
#w.duty(51)
writeulpdata(12, 6)

import time
x = readulpdata(8)
for i in range(300):
    while True:
        y = readulpdata(8)
        if y != x:
            break
    x = y
    #print(x, end=" ")
    print(readulpdata(4), end=" ")
    if (i%20) == 19:
        print()
    

13872 6212 4043 24633 7147 10190 5396 8491 6529 5588 3177 3866 5331 3423 3489 3541 4410 3725 3785 3864 
3940 4019 4095 4177 4260 4349 4435 4527 4617 4717 4813 4912 5014 5117 5224 5335 5453 5566 5694 5819 
5931 6061 6191 6336 6474 6622 6764 6920 7072 7223 7389 7558 7727 7904 8090 8282 8471 8665 8870 9087 
9305 9529 9756 10006 10256 10515 10782 8773 11285 11568 4256 10086 9737 4446 10675 12923 10523 5344 13694 14063 
13832 12556 15206 12409 15988 16461 16952 17468 18026 7313 18821 18629 778 20069 20741 21457 22202 22025 20168 24593 
25552 26593 21975 28648 29908 31282 11728 27951 34822 29118 38736 16127 40804 29172 18735 49798 51857 2305 57263 346 
2277 64295 2859 47625 7056 7251 5630 65287 14061 8182 10513 5848 9273 3209 5940 4122 7063 3651 4413 3833 
7085 4810 4128 4187 4302 7085 6372 7509 4788 6936 5030 8336 6425 6552 5600 10438 7123 9828 7622 6485 
7949 6769 11238 7214 8921 7569 7773 7956 8137 8302 8527 8754 8966 9201 9432 9666 9904 10165 10419 10688 
10961 11255 11545 11852 12168 12